# 4. 分支鏈 (Chains Branching) - Ollama 版本

本範例展示根據條件選擇不同處理路徑，建立一個智能學習助手，根據學生問題類型提供不同的學習建議。

## 學習重點
- 使用 RunnableBranch 實現條件分支
- 學習如何根據輸入內容選擇不同的處理路徑
- 理解分支邏輯的設計模式
- 掌握複雜條件判斷的實現方法


In [1]:
# 導入必要的套件
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch
from langchain_ollama.llms import OllamaLLM

# 載入環境變數
load_dotenv()

# 建立 Ollama 模型
model = OllamaLLM(model="llama3.2:latest")


In [3]:
# 定義不同學習問題類型的提示模板

# 概念解釋模板
concept_explanation_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的學習導師，擅長用簡單易懂的方式解釋複雜概念。"),
        ("human", "請解釋這個概念：{question}"),
    ]
)

# 解題指導模板
problem_solving_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的學習導師，擅長指導學生解題步驟。"),
        ("human", "請指導我如何解決這個問題：{question}"),
    ]
)

# 學習建議模板
study_advice_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的學習導師，擅長提供學習方法和建議。"),
        ("human", "請給我學習建議：{question}"),
    ]
)

# 複雜問題模板
complex_question_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的學習導師，這個問題比較複雜，需要詳細分析。"),
        ("human", "這是一個複雜問題，請詳細分析：{question}"),
    ]
)

print("學習助手模板已定義：")
print("1. concept_explanation_template: 概念解釋")
print("2. problem_solving_template: 解題指導")
print("3. study_advice_template: 學習建議")
print("4. complex_question_template: 複雜問題")


學習助手模板已定義：
1. concept_explanation_template: 概念解釋
2. problem_solving_template: 解題指導
3. study_advice_template: 學習建議
4. complex_question_template: 複雜問題


In [4]:
# 定義問題分類模板（已移除，改用簡單的字串匹配）
# 原本使用 LLM 分類會造成兩次模型調用，效率較低
# 現在直接在 RunnableBranch 的條件函數中判斷

print("準備建立分支邏輯，使用字串匹配進行問題分類")

準備建立分支邏輯，使用字串匹配進行問題分類


In [ ]:
# 定義分支處理邏輯
# 使用 RunnableBranch 根據輸入內容直接選擇不同的處理路徑
# 優化：移除了多餘的分類 LLM 調用，提升效率
# RunnableBranch的使用說明

branches = RunnableBranch(
    # 概念解釋分支 - 檢查問題中是否包含相關關鍵字
    (
        lambda x: "概念" in x.get("question", "") or "解釋" in x.get("question", "") or "什麼是" in x.get("question", ""),
        concept_explanation_template | model | StrOutputParser()
    ),
    # 解題指導分支
    (
        lambda x: "解題" in x.get("question", "") or "如何" in x.get("question", "") or "怎麼" in x.get("question", "") or "步驟" in x.get("question", ""),
        problem_solving_template | model | StrOutputParser()
    ),
    # 學習建議分支
    (
        lambda x: "學習" in x.get("question", "") or "方法" in x.get("question", "") or "建議" in x.get("question", "") or "技巧" in x.get("question", ""),
        study_advice_template | model | StrOutputParser()
    ),
    # 預設分支（複雜問題）
    complex_question_template | model | StrOutputParser()
)

print("分支處理邏輯已定義（優化版）：")
print("1. 概念問題 → 概念解釋")
print("2. 解題問題 → 解題指導")
print("3. 學習問題 → 學習建議")
print("4. 其他情況 → 複雜問題分析")
print("\n✨ 優化：直接使用字串匹配，避免多餘的 LLM 調用，提升效率！")

分支處理邏輯已定義（優化版）：
1. 概念問題 → 概念解釋
2. 解題問題 → 解題指導
3. 學習問題 → 學習建議
4. 其他情況 → 複雜問題分析

✨ 優化：直接使用字串匹配，避免多餘的 LLM 調用，提升效率！


In [7]:
# 簡化後的鏈：直接使用分支邏輯
# 移除了分類鏈，效率提升約 50%（減少一次 LLM 調用）
chain = branches

print("完整的學習助手鏈已建立（優化版）！")
print("鏈的流程：學習問題 → 字串匹配分類 → 選擇處理路徑 → 生成學習建議")
print("\n💡 效能提升：")
print("  - 原版：需要 2 次 LLM 調用（分類 + 回答）")
print("  - 優化版：只需 1 次 LLM 調用（直接回答）")

完整的學習助手鏈已建立（優化版）！
鏈的流程：學習問題 → 字串匹配分類 → 選擇處理路徑 → 生成學習建議

💡 效能提升：
  - 原版：需要 2 次 LLM 調用（分類 + 回答）
  - 優化版：只需 1 次 LLM 調用（直接回答）


In [8]:
# 測試不同類型的學習問題
test_questions = [
    "什麼是機器學習？",  # 概念解釋
    "如何解二元一次方程式？",  # 解題指導
    "有什麼好的學習方法？",  # 學習建議
    "請分析這個複雜的數學問題..."  # 複雜問題
]

print("=" * 60)
print("智能學習助手測試結果：")
print("=" * 60)

for i, question in enumerate(test_questions, 1):
    print(f"\n測試 {i}: {question}")
    print("-" * 40)
    result = chain.invoke({"question": question})
    print(f"學習建議: {result}")
    print("=" * 60)


智能學習助手測試結果：

測試 1: 什麼是機器學習？
----------------------------------------
學習建議: 好！我很高興能夠解釋給你 machine learning 的概念。

機器學習（Machine Learning）是一種技術，它使 computor 能夠自行改善和學習，以便對資料做出正確的預測或決策。

 Imagine 你有一個小孩，在玩遊戲時，越玩越好。同樣地，machine learning 就是讓機器能夠在資料上 "練習"，並且越多的資料就越好。這樣，當它遇到新的事物時，就能夠快速而正確地做出決定。

有一種常見的例子，就是facebook 的 Recommendation System。當你登入 facebook 時，它會為你推薦那些你可能會喜歡的東西。這是因為facebook 已經對你的行為和偏好有了一個很好的了解，使用machine learning 這樣的技術來幫助它做出預測。

總之，機器學習是一種能夠讓 computer 自行改善、學習和做出正確決策的技術。

測試 2: 如何解二元一次方程式？
----------------------------------------
學習建議: 能夠解二元一次方程式是學習力很強的人的特徵之一。下面提供了一個解二元一次方程式的步驟。

### 步驟一：給定方程式
你得到一個二元一次方程式，例如： $2x + 3y = 12$

### 步驟二：將方程式轉換成標準形式

將各項都帶入標準方程式的形式：

\begin{align*}ax+by&amp;=c\\ \end{align*}

其中，a、b 和 c 是常數。因此，在這個例子中： a = 2, b = 3, 和 c = 12.

### 步驟三：解 x
利用方程式的標準形式將各項相對於 x 移至一側，將各項相對於 y 移至另一側。以下是這個步驟：

\begin{align*}2x+3y&amp;=12\\ 2x-0y&amp;=-3y+12\\ \frac{2x}{1}&amp;=\frac{-3y}{1}+\frac{12}{1}\\ x&amp;=\frac{\left(-3y+12\right)}{2} \\ \end{align*}

在步驟中，我們將每個項除以 2來解出 x。

###

## 💡 重點說明

### 分支鏈的設計模式

1. **條件判斷**: 使用 lambda 函數檢查輸入內容
2. **分支處理**: 每個分支都有專門的處理邏輯
3. **預設處理**: 提供預設分支處理未分類的情況

### RunnableBranch 的使用

```python
branches = RunnableBranch(
    (條件函數1, 處理鏈1),
    (條件函數2, 處理鏈2),
    (條件函數3, 處理鏈3),
    預設處理鏈  # 最後一個是預設分支
)
```

# ### 條件函數的設計
#
# 在這裡，x 的資料類型通常是一個 Python 字典（dict），例如：{"question": "什麼是機器學習？"}
# 這是因為在 LangChain 的 chain.invoke({"question": question}) 呼叫時，會將輸入包裝成字典傳遞給鏈。
# 
# 因此，條件函數可以安全地使用 x.get("question", "") 來取得問題內容：
# - x.get("question", "")：嘗試從字典 x 取得 "question" 這個 key 的值，若不存在則回傳空字串，避免 KeyError。
#
# 條件函數範例：
# ```python
# # 條件函數應該回傳 True 或 False
# lambda x: "概念" in x.get("question", "") or "解釋" in x.get("question", "")
# ```

### ⚡ 效能優化重點

**優化前的問題**:
```python
# ❌ 效率低：需要兩次 LLM 調用
chain = classification_chain | branches
# 第1次：classification_chain 呼叫 LLM 進行分類
# 第2次：branches 中的對應鏈再次呼叫 LLM
```

**優化後的方案**:
```python
# ✅ 效率高：只需一次 LLM 調用
chain = branches
# 使用字串匹配直接分類，然後調用對應的 LLM
```

**效能提升**:
- 減少 50% 的 LLM 調用次數
- 降低約 50% 的回應時間
- 節省 API 使用成本

### 實際應用場景

- **智能學習助手**: 根據問題類型提供不同指導
- **客服系統**: 自動分類客戶問題
- **內容推薦**: 根據用戶偏好推薦內容
- **智能路由**: 根據請求類型選擇處理方式